In [10]:
import json
# import pandas as pd
# from collections import defaultdict
# from collections import Counter
# import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
# from nltk.corpus import wordnet as wn
# from nltk.tag import pos_tag
# import networkx as nx
# from numpy.random import rand, RandomState
# from nltk.corpus import stopwords
# from sklearn.cluster import KMeans
import string
# from nltk.corpus import sentiwordnet as swn
from nltk.tokenize import word_tokenize
# import matplotlib
# from mpl_toolkits.basemap import Basemap
# import geopy
# from sklearn.metrics.pairwise import linear_kernel
# from geopy.geocoders import Nominatim
# from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.stem.porter import PorterStemmer
# from nltk.stem.snowball import SnowballStemmer
# import itertools
# from sklearn.decomposition import PCA
# from sklearn.pipeline import Pipeline
# import matplotlib.pyplot as plt
# %matplotlib inline

In [11]:
def json_to_dict(filename):
    '''
    INPUT: name of json file
    OUTPUT: dictionary with city keys and description values
    take json file and return dictionary
    '''
    di = {}
    english = []
    with open('wordlist.txt') as f:
        for line in f:
            english.append(line.strip('\r\n'))
            english.append(' ')
    english = set(english)

    with open(filename) as data_file:
        data = json.load(data_file)
        for item in data:
            for key in item.keys():
                # remove punctuation, make everything lower case
                txt = ''.join(ch.lower() for ch in item[key] if ch not in set(string.punctuation))
                # remove numbers
                txt = ''.join(c for c in txt if c.isdigit() is False)
                # remove the city name from its own description
                keyparts = key.split(',')
                txt = ' '.join(c for c in txt.split() if c != keyparts[0].lower() and c != keyparts[-1].lower())
                # remove non-english words
                txt = ' '.join(c for c in txt.split() if c in english)
                # populate dictionary
                di[key.strip().strip('\n').encode('ascii', 'ignore')] = txt
    return di

In [12]:
def build_synonym_list(word_list):
    '''
    INPUT: list of words
    OUTPUT: list containing words and all of their synonyms
    take a list of words and return a list containin all synonyms of those words
    '''
    all_synonyms = []
    for word in word_list:
        synonyms = wn.synsets(word.lower())
        for s in synonyms:
            synlist = [l.name() for l in s.lemmas()]
        synlist.append(word.lower())
        all_synonyms.extend([wordnet.lemmatize(w) for w in synlist])
    return all_synonyms

In [13]:
rs_dict = json_to_dict('../scraping/ricksteves_articles_blogs_R01.json')
ta_dict = json_to_dict('../scraping/europe_city_reviews2.json')

empty_count = 0
for k in ta_dict:
    if len(ta_dict[k]) < 100:
        empty_count += 1
print empty_count

key_list = set(rs_dict.keys() + ta_dict.keys())
europe_dict = dict()
for key in key_list:
    europe_dict[key] = str(rs_dict.get(key)) + str(ta_dict.get(key))
    
# remove cities which contain little or no text
europe_dict = {key: value for key, value in europe_dict.items() if len(value) > 200}
for k in europe_dict:
    if len(europe_dict[k]) < 100:
        print '\n', k
        print europe_dict[k]

# Convert dictionary into dataframe
cities_df = pd.DataFrame.from_dict(europe_dict, orient='index', dtype=None)
cities_df.columns = ['description']

# Extract cities and their descripitions from dataframe
# doc_bodies = cities_df['description']
doc_bodies = cities_df['description'].values
tokenized_corpus = [word_tokenize(content.lower()) for content in doc_bodies]
cities = cities_df.index.values

67


In [14]:
# read each word, assign pos_tag. If pos tag is noun, append dictionary
noun_dict = defaultdict(list)
verb_dict = defaultdict(list)
adj_dict = defaultdict(list)
adv_dict = defaultdict(list)

for i, doc in enumerate(tokenized_corpus):
    pos_tag(doc)
    for word, tag in pos_tag(doc):
        if 'NN' in tag:
            noun_dict[word].append(i)
        if 'JJ' in tag:
            adj_dict[word].append(i)
        if 'VB' in tag:
            verb_dict[word].append(i)
        if 'RB' in tag:
            adv_dict[word].append(i)

NameError: name 'defaultdict' is not defined